In [3]:
import time

import numpy as np
import matplotlib as mpl
import pandas as pd

import qcodes as qc

from qcodes import Station, Parameter, ManualParameter, new_data_set, new_experiment, load_by_id
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.data_export import get_shaped_data_by_runid, get_data_by_id

__NOTE:__ this feature may not work. It is planned to make this feature work soon. However, feel free to contribute fixes for it.

This module provides a few tools for exporting data. In particular, we want to export from the v2 dataset to text files so we can use spyview. The module contains tools for exporting full datasets, and a dataset subscriber that incrementally stores data in a spyview format; this allows us to use spyview for live plotting.

In [2]:
from pytopo.qctools.dataset_export.spyview import export_dataset_to_spyview, SpyviewExporter

We use a qcodes config entry to determine where we export data to. It accepts `time.strftime` formatting strings. Here we store everyting in a year-month/year-month-day subdirectory inside the current working directory.

In [3]:
qc.config['user']['data_export_dir'] = "data/%Y-%m/%Y-%m-%d/"
qc.config.save_to_cwd()

Usage of the subscriber is pretty straight-forward. It is instantiated using the data columns that we'd like to export, here 'z' and 'z2'. Note: all specified columns must have the same dependencies ('setpoints'); i.e., using 'c' together with 'z' will raise an error. In that case we'd need to use two different exporters.

In [4]:
station = Station()
exp = new_experiment('an experiment', sample_name='not really a sample')

m = Measurement()
m.register_custom_parameter('x')
m.register_custom_parameter('y')
m.register_custom_parameter('z', setpoints=('x', 'y', ))
m.register_custom_parameter('z2', setpoints=('x', 'y', ))
m.register_custom_parameter('a')
m.register_custom_parameter('c', setpoints=('a',))


with m.run() as datasaver:
    ds = datasaver.dataset
    exporter = SpyviewExporter(ds, param_names=['z', 'z2'])
    sub_id = ds.subscribe(exporter, min_wait=0, min_count=1)
    
    for j in range(10):
        for i in range(12):
            datasaver.add_result(('x', float(i)), 
                                 ('y', float(j)), 
                                 ('z', float(i) + j**.5),
                                 ('z2', float(i)**.5 + j**.5),
                                 ('a', float(i)),
                                 ('c', i**2.))
            time.sleep(0.1)
    
    run_id = datasaver.run_id

Starting experimental run with id: 1


If we have the `run_id` of a finished experiment run, we can also export data columns to spyview compatible .dat files.

In [5]:
ds = load_by_id(ds.run_id)

export_dataset_to_spyview(ds, ['z', 'z2'])